In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''    
print(os.listdir("../input/10-monkey-species"))
print(os.listdir("../input/10-monkey-species/training"))
print(os.listdir("../input/10-monkey-species/training/training"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

In [ ]:
'''
错误：
train_dir = "../input/10-monkey-species/training"
valid_dir = "../input/10-monkey-species/validation"
label_file = "../input/10-monkey-species/monkey_labels.txt"
'''
train_dir = "../input/10-monkey-species/training/training"
valid_dir = "../input/10-monkey-species/validation/validation"
label_file = "../input/10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

In [ ]:
# labels作用：子文件夹名字 --> 真正类别  的映射
# 要是做inference， 需要转为真正的名字
labels = pd.read_csv(label_file, header=0)
print(labels)

In [ ]:
height = 224
width = 224
channels = 3
batch_size = 24
num_classes = 10

# 读取数据，再增强
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    # 专门给resnet50定义的处理图像的函数， 作用为归一化 [-1, 1]
    preprocessing_function = keras.applications.resnet50.preprocess_input, 
    # rescale = 1./255,
    rotation_range = 40,       # 生成的角度在-40~40之间，不一定为40
    width_shift_range = 0.2,   # 0-1之间的数为比例，>1为像素值
    height_shift_range = 0.2,  # 与rotation_range一样，随机指定0~20%的位移
    shear_range = 0.2,         # 剪切强度
    zoom_range = 0.2,          # 缩放强度
    horizontal_flip=True,      # 是否随机水平翻转
    fill_mode = 'nearest'      # 填充像素的规则
)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7, 
                                                   shuffle = True,
                                                   class_mode = 'categorical')
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7, 
                                                   shuffle = False,
                                                   class_mode = 'categorical')
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)
'''
异常处理：
y.shape = （64， 1）
经过one-hot编码之后应该是(64, 10)
可能原因是路径中training, validation只有一个文件夹，就将所有的图片都划分成了一个类
解决：在进一步打开文件夹
'''

In [ ]:
resnet50_fine_tune = keras.models.Sequential()
resnet50_fine_tune.add(keras.applications.ResNet50(include_top = False, 
# ResNet50针对ImageNet训练，有1000类， 我们的问题只有10类，所以需要把最后一层给去掉，使能起迁移的作用
                                                   pooling = 'avg',
# ResNet50倒数第二层是一个卷积层的输出(三维的矩阵，而不是向量)，想要连接FC，1）pooling 2）展平                                         
                                                   weights = 'imagenet'))
# None:从头开始训练；imagenet:去下载一个已经在ImageNet上训练好的模型中的参数


# 刚才去掉了最后一层，现在需要加上自己任务的全连接层
resnet50_fine_tune.add(keras.layers.Dense(num_classes, activation = 'softmax'))

# 设定trainable，固定下载的模型
# resnet50_fine_tune将已有的50层作为一层，所以只要设置一次
resnet50_fine_tune.layers[0].trainable = False

resnet50_fine_tune.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',  # fine_tune适合设置为'sgd'
                           metrics = ['accuracy'])
resnet50_fine_tune.summary()

In [ ]:
epochs = 10
history = resnet50_fine_tune.fit_generator(train_generator, 
                                           steps_per_epoch = train_num // batch_size,
                                           epochs = epochs,
                                           validation_data = valid_generator,
                                           validation_steps = valid_num // batch_size)

In [ ]:
print(history.history.keys())

In [ ]:
def plot_learning_curve(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_' + label] = history.history['val_' + label]
    pd.DataFrame(data).plot(figsize = (8, 5))
    plt.grid()
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curve(history, 'loss', epochs, 0, 2)
plot_learning_curve(history, 'accuracy', epochs, 0, 1)

In [ ]:
# fine_tune ResNet50层中的最后几层
resnet50 = keras.applications.ResNet50(include_top = False, 
                                                   pooling = 'avg',                                        
                                                   weights = 'imagenet')
resnet50.summary()


In [ ]:
# 对后除了五层以外所有层次遍历
for layer in resnet50.layers[0: -5]:
    layer.trainable = False
    
# 组合
resnet50_new = keras.models.Sequential([
    resnet50,
    keras.layers.Dense(num_classes, activation = 'softmax')
])

resnet50_new.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',  # fine_tune适合设置为'sgd'
                           metrics = ['accuracy'])
resnet50_new.summary()

In [ ]:
epochs = 10
history = resnet50_new.fit_generator(train_generator, 
                                     steps_per_epoch = train_num // batch_size,
                                     epochs = epochs,
                                     validation_data = valid_generator,
                                     validation_steps = valid_num // batch_size)
# 结果可能变差：
# 1）可训练参数变多，需要更多的epoch来达到更好的效果
# 2）trainbale层中游一部分为ResNet原来的层，以及新加的全连接层。
# 原来的层i)通过前面很多层训练非常出色的权重没有了  ii)需要配合新的层的学习率，导致原本的能力没有发挥